In [ ]:
import cv2
import mtcnn
import numpy as np
from keras_vggface import VGGFace
from scipy.spatial.distance import cosine
from keras_vggface.utils import preprocess_input, decode_predictions

detector = mtcnn.MTCNN()
model = VGGFace(model="resnet50")

def main():

    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)

    while 1:
        key = cv2.waitKey(100)
    
        if key == 27:
            break
            
        elif key == 32:
            faces = get_faces(frame)
            y_ref = recognize_faces(faces, frame)
            draw_boxes(frame, faces, "Reference")
            
        elif key == 9:
            faces = get_faces(frame)
            y_hat = recognize_faces(faces, frame)
            index = verify_face(y_hat, y_ref)
            draw_boxes(frame, faces, "Verification", index)

        ret, frame = cap.read()
        cv2.imshow('Webcam', frame)
            
    cap.release()
    cv2.destroyAllWindows()


def get_faces(frame):
    
    shot = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    return detector.detect_faces(shot)
    
    
def recognize_faces(faces, frame):
    samples = np.zeros((len(faces), 224, 224, 3), dtype='float32')
    
    for i, face in enumerate(faces):
        x1, y1, w, h = face["box"]
        x2, y2 = x1 + w, y1 + h
            
        roi = frame[y1:y2, x1:x2]
        roi_sized = cv2.resize(roi, (224, 224))
        samples[i] = roi_sized
        
    samples_prep = preprocess_input(samples)
    return model.predict(samples_prep)

    
def verify_face(y_hat, y_ref):
    
    number_faces = y_hat.shape[0]
    scores = np.zeros(number_faces)
    for i in range(number_faces):
        scores[i] = cosine(y_hat[i], y_ref[0])
    
    
    index = np.argmin(scores)
    print(index)
    return index
    
    
def draw_boxes(frame, faces, window_name, face_index = -1):
    for i, face in enumerate(faces):
        print(i)
        color = (0, 0, 255)
        
        if i == face_index:
            color = (0, 255, 0)
        
        x1, y1, w, h = face["box"]
        x2, y2 = x1 + w, y1 + h
        boxed = cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.imshow(window_name, boxed)

main()

In [ ]:
    cap.release()
    cv2.destroyAllWindows()